## Libraries required

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt

## Downloading the webpage

In [6]:
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.3619&lon=-121.9288#.XBbssFVKjek")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Tonight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Tonight: Mostly cloudy, with a low around 48. North wind around 6 mph becoming calm  in the evening. " class="forecast-icon" src="newimages/medium/nbkn.png" title="Tonight: Mostly cloudy, with a low around 48. North wind around 6 mph becoming calm  in the evening. "/>
 </p>
 <p class="short-desc">
  Mostly Cloudy
 </p>
 <p class="temp temp-low">
  Low: 48 °F
 </p>
</div>


## To get weather update for today

In [8]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()

print(period)
print(short_desc)
print(temp)

Tonight
Mostly Cloudy
Low: 48 °F


## To get the detailed udpate for today

In [9]:
img = tonight.find("img")
desc = img['title']

print(desc)

Tonight: Mostly cloudy, with a low around 48. North wind around 6 mph becoming calm  in the evening. 


## To get period of coming 7 days

In [11]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Tonight',
 'Friday',
 'FridayNight',
 'Saturday',
 'SaturdayNight',
 'Sunday',
 'SundayNight',
 'Monday',
 'MondayNight']

## To get temperature, short and long description of coming 7 days

In [13]:
short_descs = [sd.get_text() 
               for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() 
         for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] 
         for d in seven_day.select(".tombstone-container img")]

print(short_descs)
print(temps)
print(descs)

['Mostly Cloudy', 'Partly Sunny', 'Partly Cloudy', 'Mostly Sunny', 'Partly Cloudy', 'Partly Sunny', 'Mostly Cloudy', 'Chance Rain', 'ChanceShowers']
['Low: 48 °F', 'High: 60 °F', 'Low: 42 °F', 'High: 59 °F', 'Low: 44 °F', 'High: 61 °F', 'Low: 46 °F', 'High: 59 °F', 'Low: 46 °F']
['Tonight: Mostly cloudy, with a low around 48. North wind around 6 mph becoming calm  in the evening. ', 'Friday: Partly sunny, with a high near 60. Calm wind becoming north northwest 5 to 7 mph in the morning. ', 'Friday Night: Partly cloudy, with a low around 42. West northwest wind 5 to 11 mph. ', 'Saturday: Mostly sunny, with a high near 59. Northwest wind around 11 mph. ', 'Saturday Night: Partly cloudy, with a low around 44. North northwest wind 3 to 8 mph. ', 'Sunday: Partly sunny, with a high near 61.', 'Sunday Night: Mostly cloudy, with a low around 46.', 'Monday: A chance of rain.  Mostly cloudy, with a high near 59.', 'Monday Night: A chance of showers.  Mostly cloudy, with a low around 46.']


## Spreadsheet view of 7 Day weather forecast

In [20]:
weather = pd.DataFrame({
        "Period": periods, 
        "Short Description": short_descs, 
        "Temperature": temps, 
        "Description":descs
    })
weather

,Period,Short Description,Temperature,Description
0,Tonight,Mostly Cloudy,Low: 48 °F,"Tonight: Mostly cloudy, with a low around 48. ..."
1,Friday,Partly Sunny,High: 60 °F,"Friday: Partly sunny, with a high near 60. Cal..."
2,FridayNight,Partly Cloudy,Low: 42 °F,"Friday Night: Partly cloudy, with a low around..."
3,Saturday,Mostly Sunny,High: 59 °F,"Saturday: Mostly sunny, with a high near 59. N..."
4,SaturdayNight,Partly Cloudy,Low: 44 °F,"Saturday Night: Partly cloudy, with a low arou..."
5,Sunday,Partly Sunny,High: 61 °F,"Sunday: Partly sunny, with a high near 61."
6,SundayNight,Mostly Cloudy,Low: 46 °F,"Sunday Night: Mostly cloudy, with a low around..."
7,Monday,Chance Rain,High: 59 °F,"Monday: A chance of rain. Mostly cloudy, with..."
8,MondayNight,ChanceShowers,Low: 46 °F,Monday Night: A chance of showers. Mostly clo...


## To save the 7 day forecast in json file

In [ ]:
to export the pandas table to json file
json_out = weather.to_json(orient='table')
with open('weather_report.json', 'w') as f:
   f.write(json_out)

## To get only temparture value of 7 day forecast

In [31]:
temp_nums = weather["Temperature"].str.extract("(?P<temp_num>\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    48
1    60
2    42
3    59
4    44
5    61
6    46
7    59
8    46
Name: temp_num, dtype: object

## To get mean temperature of 7 days forecast

In [35]:
weather["temp_num"].mean()

51.666666666666664

## To check day or night forecast(True signifies night time)

In [36]:
is_night = weather["Temperature"].str.contains("Low")
weather["is_night"] = is_night
is_night

0     True
1    False
2     True
3    False
4     True
5    False
6     True
7    False
8     True
Name: Temperature, dtype: bool

## To view weather forecast of night time

In [37]:
weather[is_night]

,Period,Short Description,Temperature,Description,temp_num,is_night
0,Tonight,Mostly Cloudy,Low: 48 °F,"Tonight: Mostly cloudy, with a low around 48. ...",48,True
2,FridayNight,Partly Cloudy,Low: 42 °F,"Friday Night: Partly cloudy, with a low around...",42,True
4,SaturdayNight,Partly Cloudy,Low: 44 °F,"Saturday Night: Partly cloudy, with a low arou...",44,True
6,SundayNight,Mostly Cloudy,Low: 46 °F,"Sunday Night: Mostly cloudy, with a low around...",46,True
8,MondayNight,ChanceShowers,Low: 46 °F,Monday Night: A chance of showers. Mostly clo...,46,True
